# Multi-armed bandits

### Intro.

강화학습에서는 학습정보를 통해 올바른 action을 지시(instruct)하기보다는 선택가능한 action들에 대한 가치를 평가(evaluate)한다. 이것이 바로 강화학습을 다른 종류의 학습과 구분짓은 가장 중요한 특징이다. 이번 챕터에서는 일명 k-armed bandit problem이라고 불리는, 오직 하나의 state에서 독립된 k개의 action을 선택하는, 특수한 강화학습 문제를 살펴보도록 한다. 또한 이를 통해 몇 가지 solution methods를 살펴보도록 할 것이다.

### k-armed bandit problem

'One-armed bandit'이라는 말을 들어본 적이 있는가? 단어 그대로 해석하면 외팔이 강도라는 뜻인데, 이는 도박용 슬롯머신의 생김새를 본따 비꼬아 표현한 말이다.

<img src="images/one-armed_bandit.png" alt="One-armed bandits" style="width: 600px;"/>
<Center>[Fig. 1] One-armed bandit (['one-armed bandit' searched by google](https://www.google.co.kr/search?q=one-armed+bandit&tbm=isch))</Center>

그렇다면 k-armed bandit이란 무엇일까? 이는 마치 k개의 슬롯머신이 내 눈 앞에 있는 것과 같은 상황을 묘사한 문제라고 볼 수 있다. 

<img src="images/multi-armed_bandit.jpg" alt="Multi-armed bandits" style="width: 300px;"/>
<Center>[Fig. 2] Multi-armed bandit ([Microsoft research](https://www.microsoft.com/en-us/research/))</Center>

아주 간단한 형태의 k-armed bandit problem을 다음과 같이 정의해보자.

1. k개의 bandit machine 중에서 하나를 선택한다. 이는 매 time step마다 k개의 액션을 선택할 수 있는 것과 같다.
2. 선택된 bandit machine은 고정된 확률 분포(stationary probability distribution)를 통해 수치적인(numerical) reward를 반환한다.

우리의 목표는 위 절차를 어떤 time step만큼 반복하여 reward의 총합에 대한 기댓값이 가장 높은 bandit machine을 찾는 것이다. 시간이 흐르고 action을 반복하여 시행할수록, 우리는 점차 보상을 최대화 할 수 있으리라 여겨지는 action(the best lever)에 집중하게 될 것이다. 

참고로 오늘날의 'bandit problem'은 위의 예시 뿐만 아니라, action의 선택과 그에 대한 reward의 반환이 반복되는 문제를 총체적으로 일컫는다. 

### References

1. something